In [ ]:
import sys
sys.path.insert(1, '../')
import analyze_kinetics
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### import data 27C

In [ ]:
sorption_list = {}

path_dictionary = {'ROS-037':'../../data/ROS-037/27C/',
                   'ROS-039':'../../data/ROS-039/27C/',
                   'ROS-040':'../../data/ROS-040/27C/',
                   'MOF-303':'../../data/MOF-303/27C/',
                   'MIL-160':'../../data/MIL-160/27C/',
                   'Syloid':'../../data/Syloid/27C/',
                   'CAU-10':'../../data/CAU-10/27C/',
                   'Al-fumarate':'../../data/Al-fumarate/27C/',
                   }

for material in path_dictionary.keys():
    path = path_dictionary[material]
    filelist = os.listdir(path)
    filelist = [i for i in filelist if (i[-4:]=='.xls') or (i[-5:]=='.xlsx')]
    sorption_list[material] = {'data':[], 
                                'fitting_result':pd.DataFrame(columns=['sample_mass','experiment',  'cycle', 'popt', 'pcov', 'R2', 'RH_target'], dtype=object),
                                'k':pd.DataFrame(columns=['k','sigma',  ], dtype=object),
                              }
    
    for filename in filelist:
        print(filename)
        f = analyze_kinetics.Sorption()
        f.read_file(path, filename)
        print(f.filename)
        print(f.experiment_type)
        sorption_list[material]['data'].append(f)
        if f.experiment_type == 'isotherm':
            iso = analyze_kinetics.Isotherm()
            iso.extract_isotherm(f)
            iso.interpolate_uptake_to_RH_ads()
            iso.interpolate_uptake_to_RH_des()
            iso.interpolate_RH_to_uptake_ads()
            iso.interpolate_RH_to_uptake_des()
            sorption_list[material]['isotherm'] = iso
            plt.plot(iso.RHtarget_ads, iso.adsorption)
            plt.plot(iso.RHtarget_des, iso.desorption)
            plt.show()
            plt.plot(np.linspace(min(iso.adsorption), max(iso.adsorption), 100), 
                     iso.uptake_to_RH_ads(np.linspace(min(iso.adsorption), max(iso.adsorption), 100)))
            #plt.plot(i.RHtarget_des, i.desorption)
            plt.show()
        else:
            kin = analyze_kinetics.Kinetics()
            kin.decompose_to_cycles(f)
            #sorption_list['ROS-037']['kinetics'] = kin
            #sorption_list['ROS-037'].append(f)
            plt.plot(f.data.time, f.data.uptake)
            plt.plot(f.data.time, f.data.cycle_number)
            plt.show()

## Figures S41-S97

In [ ]:
for material in path_dictionary.keys():
    for sorption in sorption_list[material]['data']:
        if sorption.experiment_type == 'kinetics':
            kin = analyze_kinetics.Kinetics()
            iso = sorption_list[material]['isotherm']
            kin.fit_model2(sorption, 2, 90, iso, material, plot=True, verbose = True, scale_isotherm = True)
            sorption_list[material]['fitting_result'] = sorption_list[material]['fitting_result'].append(kin.result)
            print(kin.result)
            


## Figures S99-S100

In [ ]:
title_dictionary = {0: 'Desorption', 30:'Adsorption 30 %RH', 60:'Adsorption 60 %RH',}
def fit_mass(result, i, plot = True, sigma = True)   :     
    from scipy.optimize import curve_fit
    from sklearn.metrics import r2_score
    from scipy.stats.distributions import t
    def line_zero(x, slope):
        return slope*x 

    print(result)
    x = 1/result.sample_mass
    y = result.popt
    if sigma:
        sigma_values = result.pcov**0.5
        popt, pcov = curve_fit(line_zero, x, y, [ 1], sigma = sigma_values,#maxfev=15000,
                          )
    else:
        popt, pcov = curve_fit(line_zero, x, y, [ 1], #maxfev=15000,
                          )
    print(popt, pcov[0, 0]**0.5)

    alpha = 0.05 
    n = len(list(set(result.experiment.values)))
    p = 1
    dof = max(0, n - p) 
    tval = t.ppf(1.0 - alpha / 2.0, dof) 
    print(tval*pcov[0, 0]**0.5)
    
    if plot:
        arrax[i].plot(x, y, 'o', label = 'k')
        arrax[i].plot(np.linspace(0, max(x), 100), line_zero(np.linspace(0, max(x), 100), *popt), '-', label='best fit')
        #arrax[i].plot(np.linspace(0, max(x), 100), line_zero(np.linspace(0, max(x), 100), popt[0]+pcov[0, 0]**0.5), 'g--', label='standard deviation')
        #arrax[i].plot(np.linspace(0, max(x), 100), line_zero(np.linspace(0, max(x), 100), popt[0]-pcov[0, 0]**0.5), 'g--', )
        arrax[i].legend()
        arrax[i].set_title(title_dictionary[RH_target])
        arrax[i].set_ylabel('k, wt.%/min/%RH')
        arrax[i].set_xlabel('1/mass, 1/mg')
        
    return pd.DataFrame({'k':popt[0], 
                         'k_sigma':pcov[0, 0]**0.5, 
                         'k_CI':tval*pcov[0, 0]**0.5,
                         'RH_target':RH_target, 
                         'R2_threshold':R2_threshold, 
                         'sigma':sigma,
                         },  index=[0], dtype=object)

R2_threshold = 0.9
for material in path_dictionary.keys():       
    sorption_list[material]['k'] = pd.DataFrame(columns=['k','sigma',  ], dtype=object)
    fig, arrax = plt.subplots(1, 3, figsize = (10, 3))
    for i, RH_target in enumerate([ 0, 30, 60]):
        result = sorption_list[material]['fitting_result']
        result = result[(result.RH_target==RH_target)&(result.R2>R2_threshold)]
        sorption_list[material]['k'] = sorption_list[material]['k'].append(fit_mass(result, i,
                                                                                                     plot = True, 
                                                                                                     sigma = False),
                                                                           ignore_index=True)
    suptitle = plt.suptitle(material,y=1.02 )
    plt.tight_layout()
    plt.savefig('K_fitting{0}.png'.format(material, 
                                     ), bbox_extra_artists=(suptitle,), bbox_inches="tight")
    plt.show()


In [ ]:
for material in list(path_dictionary.keys()):
    print(sorption_list[material]['fitting_result'])

In [ ]:
import json

with open('sorption_fitting_results_27C.json', 'w') as outfile:
    json.dump({material:[sorption_list[material]['fitting_result'].to_dict(),sorption_list[material]['k'].to_dict()]  for material in list(sorption_list.keys())}, outfile)

## Figure S101

In [ ]:
import json

with open('sorption_fitting_results_27C.json', 'r') as infile:
    data = json.load(infile)
    print(data)

In [ ]:
title_dictionary = {'0': 'Desorption', '1':'Adsorption 30 %RH', '2':'Adsorption 60 %RH',}
material_list = [ 'ROS-037', 'ROS-039', 'ROS-040', 'MOF-303', 'MIL-160', 'Syloid', 'CAU-10', 'Al-fumarate']
for RH_target in [ '0', '1',  '2']:
    plt.bar(material_list, 
        [data[material][1]['k'][RH_target] for material in material_list], 
        yerr=[data[material][1]['k_sigma'][RH_target] for material in material_list], 
        align='center', alpha=0.5, ecolor='black', capsize=10)
    plt.title(title_dictionary[RH_target])
    plt.ylabel('k\', (wt.%·mg)/(min·%RH)')
    plt.xticks(rotation=90)
    plt.show()

## Calculate heatmaps

In [ ]:
def working_capacity_predict( t1, t2, Kads, Kdes, RH_target, Isotherm, verbose=False, plot=False):
    t0 = 0
    w0 = Isotherm.RH_to_uptake_ads(RH_target)
    tolerance = 0.001
    work_capacity_all = []
    def curve( w0,  t1, K, t_range, RH):
        uptake_out = []
        B = w0
        t0 = t_range[0]
        for i, t in enumerate(t_range):
            B = B + (t-t0)* K* (RH - Isotherm.uptake_to_RH_ads(B))   
            uptake_out.append(B)
            t0=t
        return uptake_out

    w_last = w0
    workcapacity_ads_last = 0
    uptake_des_last = 0
    equilibrium_cycle = 0
    for i in range(1000):
        t_range = np.linspace(0+i*(t1+t2), t1+i*(t1+t2), num=100)
        w_range = curve( w_last,  t1, Kads, t_range, RH_target)
        w_last = w_range[-1]
        work_capacity = w_last
        #plt.scatter(t_range[-1],w_last)
        if plot: plt.plot(t_range, w_range, c='C0')

        t_range = np.linspace(t1+i*(t1+t2), (t1+t2)+i*(t1+t2), num=100)
        w_range = curve(w_last,  t2, Kdes, t_range, 0)
        w_last = w_range[-1]
        work_capacity = work_capacity - w_last
        #plt.scatter(t_range[-1],w_last)
        if plot: plt.plot(t_range, w_range, c='C0')

        work_capacity_all.append(work_capacity)
        if (abs(uptake_des_last - w_range[-1]) <tolerance)&(abs(workcapacity_ads_last-work_capacity) < tolerance):
            equilibrium_cycle = i
            break
        #print(work_capacity, work_capacity/(t1+t2))
        uptake_des_last = w_range[-1]
        workcapacity_ads_last = work_capacity
        ## plot (unnest x and y)

    if plot:print(work_capacity_all[-1]/(t1+t2))
    if plot:plt.ylabel('Uptake (wt.%)')
    if plot:plt.xlabel('Time (min)')
    #if plot:plt.text(0,11,'t_ads={3:.2f} min, t_des={4:.2f} min,\na={0:.2f}%, k_ads={1}, k_des={2},\nN_cycles={5}, time to eq ={7:.1f}min, tolerance={6}wt.%'.format(C0,k1,k2,t1,t2, equilibrium_cycle, tolerance, equilibrium_cycle*(t1+t2)), horizontalalignment='left', fontsize=12)
    if plot:plt.show()
    return work_capacity_all[-1]/(t1+t2), equilibrium_cycle, equilibrium_cycle*(t1+t2)


In [ ]:
for material in path_dictionary.keys():       
    for RH_target in [ 30]:
        number_of_steps = 100
        t_ads_range = np.linspace(1, 100, num=number_of_steps)
        t_des_range = np.linspace(1, 100, num=number_of_steps)

        zero_mass = 10
        Kads = sorption_list[material]['k'][(sorption_list[material]['k'].RH_target == RH_target)].k.values[0]/zero_mass
        Kdes = sorption_list[material]['k'][(sorption_list[material]['k'].RH_target == 0)].k.values[0]/zero_mass

        working_capacity_per_time_array = np.zeros([number_of_steps, number_of_steps])
        equilibrium_cycle_array = np.zeros([number_of_steps, number_of_steps])
        time_to_equilibrium_array = np.zeros([number_of_steps, number_of_steps])

        for i, t1 in enumerate(t_ads_range):
            print(t1)
            for j, t2 in enumerate(t_des_range):
                working_capacity_per_time_array[i,j], equilibrium_cycle_array[i,j], time_to_equilibrium_array[i,j] = \
                working_capacity_predict(t1, t2, 
                                          Kads, Kdes,RH_target,
                                          sorption_list[material]['isotherm'], plot = False)

        print('Max working capacity per min: ', working_capacity_per_time_array.max())

        ind_max = np.unravel_index(np.argmax(working_capacity_per_time_array, axis=None), working_capacity_per_time_array.shape)
        fig,ax=plt.subplots(1,1)
        cp = ax.contourf(t_des_range, t_ads_range, working_capacity_per_time_array, cmap='hsv', alpha=0.4)
        #cp = ax.contourf(t_des_range, t_ads_range, working_capacity_per_time_array,  vmin=0)
        fig.colorbar(cp) # Add a colorbar to a plot
        ax.set_title('Working capacity per time (wt.%/min)')
        ax.set_xlabel('t desorption (min)')
        ax.set_ylabel('t adsorption (min)')
        plt.scatter(t_des_range[ind_max[1]],t_ads_range[ind_max[0]], s=100, marker='+', c='r')
        #plt.plot(t_des_range, t_des_range*k2/k1, label = 'k des/k ads')
        #plt.legend() 
        plt.show()

        np.savetxt('Heatmap_{0}_ads{1}-{2}_des{3}-{4}_n{6}_{5}RH.csv'.format(material, 
                                                                        min(t_ads_range),
                                                                        max(t_ads_range),
                                                                        min(t_des_range),
                                                                        max(t_des_range),
                                                                        RH_target,
                                                                        number_of_steps
                                                                        ), working_capacity_per_time_array, delimiter=',')

        print('Optimal cycle:')
        print('adsorption: {:.1f} min'.format(t_ads_range[ind_max[0]]))
        print('desorption: {:.1f} min'.format(t_des_range[ind_max[1]]))
        working_capacity_predict(t_ads_range[ind_max[0]], t_des_range[ind_max[1]], 
                                 Kads, Kdes,RH_target,
                                sorption_list[material]['isotherm'], plot = True)
        working_capacity_predict(15,15, 
                                 Kads, Kdes,RH_target,
                                sorption_list[material]['isotherm'], plot = True)